# Introduction

In the tutorial, you learned how to define a simple heuristic that the agent used to select moves.  In this tutorial, you'll check your understanding and make the heuristic more complex.

To get started, run the code cell below to set up our feedback system.

In [ ]:
from learntools.core import binder
binder.bind(globals())
from learntools.rl.ex2 import *

### 1) Which move does the agent select?

Consider the board below, and say it's the red player's turn.

[insert image here]

If the player uses the heuristic from the tutorial (also shown below), which column does it select?  

<center>
<img src="https://i.imgur.com/UlUYhgr.png" width=50%><br/>
</center>

Fill in the blank below.  Your answer should be an integer between 0 and 6, inclusive.

In [ ]:
# Your code here
selected_column = ____

# Check your answer
q_1.check()

In [ ]:
#%%RM_IF(PROD)%%
selected_column = ____
q_1.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_1.hint()
#_COMMENT_IF(PROD)_
q_1.solution()

### 2) Choosing weights

In the tutorial, you saw an agent that 

In [ ]:
# Your code here: ...

# Check your answer
q_2.check()

In [ ]:
#%%RM_IF(PROD)%%

q_2.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_2.hint()
#_COMMENT_IF(PROD)_
q_2.solution()

### 3) Looking ahead

So far, you have encoded an agent that always selects the winning move, if it's available.  And, if

Will this agent always win (or draw) the game?  If not, why not?

Will this produce an optimal agent?  Or, will the agent have room for improvement?

In [ ]:
#_COMMENT_IF(PROD)_
q_3.hint()

In [ ]:
#_COMMENT_IF(PROD)_
q_3.solution()

# Keep going

Move on to **[develop a longer-term strategy](#$NEXT_NOTEBOOK_URL$)** with the minimax algorithm.